<a href="https://colab.research.google.com/github/nkhar21/ML_4/blob/main/Facial_Recognition_Basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0.1 Kaggle

In [ ]:
 !pip install kaggle

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nkhar21","key":"5be6dca82ee543f5a1e380f0fccfe96d"}'}

In [ ]:
!mkdir -p ~/.kaggle/
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge

 82% 234M/285M [00:00<00:00, 623MB/s] 
100% 285M/285M [00:00<00:00, 645MB/s]


In [ ]:
!unzip challenges-in-representation-learning-facial-expression-recognition-challenge.zip

Archive:  challenges-in-representation-learning-facial-expression-recognition-challenge.zip
  inflating: example_submission.csv  
  inflating: fer2013.tar.gz          
  inflating: icml_face_data.csv      
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd

example_submission_df = pd.read_csv('example_submission.csv')
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')
face_data_df = pd.read_csv('icml_face_data.csv')

In [ ]:
# test_df.head()

,pixels
0,254 254 254 254 254 249 255 160 2 58 53 70 77 ...
1,156 184 198 202 204 207 210 212 213 214 215 21...
2,69 118 61 60 96 121 103 87 103 88 70 90 115 12...
3,205 203 236 157 83 158 120 116 94 86 155 180 2...
4,87 79 74 66 74 96 77 80 80 84 83 89 102 91 84 ...


In [ ]:
train_df.head()

,emotion,pixels
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [ ]:
example_submission_df.head()

,3
0,4
1,0
2,4
3,3
4,3


In [ ]:
face_data_df.head()

,emotion,Usage,pixels
0,0,Training,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1,0,Training,151 150 147 155 148 133 111 140 170 174 182 15...
2,2,Training,231 212 156 164 174 138 161 173 182 200 106 38...
3,4,Training,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4,6,Training,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [ ]:
import numpy as np

# Convert pixel strings to numpy arrays
def process_pixels(pixels_str):
    return np.array(pixels_str.split(), dtype='float32')

# For training
train_df['pixels'] = train_df['pixels'].apply(process_pixels)
x_train = np.stack(train_df['pixels'].values)
y_train = train_df['emotion'].values

# For testing
test_df['pixels'] = test_df['pixels'].apply(process_pixels)
x_test = np.stack(test_df['pixels'].values)

In [ ]:
# Normalize and reshape
x_train = x_train / 255.0
x_test = x_test / 255.0

x_train = x_train.reshape(-1, 48, 48, 1)
x_test = x_test.reshape(-1, 48, 48, 1)

In [ ]:
x_test.shape, x_train.shape, y_train.shape

((7178, 48, 48, 1), (28709, 48, 48, 1), (28709,))

In [ ]:
emotion_dict = {
    0: "Angry",
    1: "Disgust",
    2: "Fear",
    3: "Happy",
    4: "Sad",
    5: "Surprise",
    6: "Neutral"
}

plt.imshow(x_train[idx].squeeze(), cmap='gray')
plt.title(f'Emotion: {emotion_dict[y_train[idx]]}')
plt.axis('off')
plt.show()


NameError: name 'x_train' is not defined

# 0.2 Wandb

In [ ]:
!pip install wandb
import wandb
wandb.login()


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: nkhar21 (nkhar21-student) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
WANDB_API_KEY  = 'f8a227b42dc881e037b25911fa86b8a491fc0581'

In [ ]:
wandb.init(project="ML_4", entity="nkhar21-student", name="test_run")

In [ ]:
wandb.log({"Hello": 1, "Test_log": 1})

In [ ]:
wandb.finish()

Hello,▁
Test_log,▁
Hello,1
Test_log,1


In [ ]:
input_dim = 48 * 48
output_dim = 7
hidden_layer_size = 48 * 4


from sklearn.model_selection import train_test_split

# Split into train and validation sets (e.g. 80% train, 20% val)
x_train, x_val, y_train, y_val = train_test_split(
    x_train, y_train, test_size=0.2, random_state=42, stratify=y_train
)

x_train.shape, x_test.shape, y_train.shape, x_val.shape, y_val.shape

((22967, 48, 48, 1), (7178, 48, 48, 1), (22967,), (5742, 48, 48, 1), (5742,))

# 1.0 Basic FC Training

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb

# === 1. WANDB INIT ===
wandb.init(project="ML_4", entity="nkhar21-student", name="First_Basic_FC")

# === 2. CONFIG ===
batch_size = 64
epochs = 10
learning_rate = 0.01
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === 3. CONVERT TO TENSORS ===
def prepare_tensor(x, y):
    x_tensor = torch.tensor(x, dtype=torch.float32).permute(0, 3, 1, 2)  # (N, 1, 48, 48)
    y_tensor = torch.tensor(y, dtype=torch.long)
    return x_tensor, y_tensor

x_train_tensor, y_train_tensor = prepare_tensor(x_train, y_train)
x_val_tensor, y_val_tensor = prepare_tensor(x_val, y_val)
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).permute(0, 3, 1, 2)  # just to save for prediction later

# === 4. DATALOADERS ===
train_loader = DataLoader(TensorDataset(x_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(TensorDataset(x_val_tensor, y_val_tensor), batch_size=batch_size)

# === 5. MODEL ===
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(input_dim, hidden_layer_size),
    nn.ReLU(),
    nn.Linear(hidden_layer_size, output_dim),
)
model.to(device)
wandb.watch(model)

# === 6. LOSS + OPTIMIZER ===
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# === 7. TRAINING LOOP ===
for epoch in range(epochs):
    model.train()
    train_loss, train_correct, train_total = 0, 0, 0

    for batch_x, batch_y in train_loader:
        batch_x, batch_y = batch_x.to(device), batch_y.to(device)
        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = loss_fn(outputs, batch_y)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        train_correct += (preds == batch_y).sum().item()
        train_total += batch_y.size(0)

    train_acc = train_correct / train_total
    avg_train_loss = train_loss / len(train_loader)

    # === VALIDATION ===
    model.eval()
    val_loss, val_correct, val_total = 0, 0, 0
    with torch.no_grad():
        for batch_x, batch_y in val_loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)
            loss = loss_fn(outputs, batch_y)
            val_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            val_correct += (preds == batch_y).sum().item()
            val_total += batch_y.size(0)

    val_acc = val_correct / val_total
    avg_val_loss = val_loss / len(val_loader)

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"  Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Val   Loss: {avg_val_loss:.4f} | Val   Acc: {val_acc:.4f}")
    print(f"  Test  Samples: {x_test_tensor.shape[0]} (held out for prediction)")

    # === LOG TO WANDB ===
    wandb.log({
        "epoch": epoch + 1,
        "train_loss": avg_train_loss,
        "train_accuracy": train_acc,
        "val_loss": avg_val_loss,
        "val_accuracy": val_acc,
        "test_samples": x_test_tensor.shape[0],
    })

# === 8. SAVE MODEL ===
torch.save(model.state_dict(), "basic_model.pth")
wandb.save("basic_model.pth")


Epoch 1/10
  Train Loss: 1.8092 | Train Acc: 0.2543
  Val   Loss: 1.7821 | Val   Acc: 0.2682
  Test  Samples: 7178 (held out for prediction)
Epoch 2/10
  Train Loss: 1.7699 | Train Acc: 0.2787
  Val   Loss: 1.7528 | Val   Acc: 0.2891
  Test  Samples: 7178 (held out for prediction)
Epoch 3/10
  Train Loss: 1.7387 | Train Acc: 0.3072
  Val   Loss: 1.7315 | Val   Acc: 0.3370
  Test  Samples: 7178 (held out for prediction)
Epoch 4/10
  Train Loss: 1.7112 | Train Acc: 0.3298
  Val   Loss: 1.6909 | Val   Acc: 0.3521
  Test  Samples: 7178 (held out for prediction)
Epoch 5/10
  Train Loss: 1.6928 | Train Acc: 0.3404
  Val   Loss: 1.7056 | Val   Acc: 0.3326
  Test  Samples: 7178 (held out for prediction)
Epoch 6/10
  Train Loss: 1.6793 | Train Acc: 0.3443
  Val   Loss: 1.6628 | Val   Acc: 0.3565
  Test  Samples: 7178 (held out for prediction)
Epoch 7/10
  Train Loss: 1.6675 | Train Acc: 0.3482
  Val   Loss: 1.6651 | Val   Acc: 0.3617
  Test  Samples: 7178 (held out for prediction)
Epoch 8/10
  

['/content/wandb/run-20250530_161151-4rfcbad1/files/basic_model.pth']

# 1.1 Training FC 2

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb

# === 1. WANDB INIT ===
wandb.init(project="ML_4", entity="nkhar21-student", name="Decomposed_Eval_Model")

# === 2. CONFIG ===
batch_size = 64
epochs = 10
learning_rate = 0.01
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === 3. TEST TENSOR (for prediction later) ===
x_test_tensor = torch.tensor(x_test, dtype=torch.float32).permute(0, 3, 1, 2)

# === 4. MODEL ===
input_dim = 48 * 48
hidden1 = 48*4
hidden2 = 48
hidden3 = 28  # 7*4
output_dim = 7

model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(input_dim, hidden1),
    nn.ReLU(),
    nn.Linear(hidden1, hidden2),
    nn.ReLU(),
    nn.Linear(hidden2, hidden3),
    nn.ReLU(),
    nn.Linear(hidden3, output_dim),
)
model.to(device)
wandb.watch(model)

# === 5. LOSS + OPTIMIZER ===
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# === 6. EVALUATION FUNCTION ===
def evaluate_model(x, y, model, loss_fn, device, batch_size=64, train_mode=False):
    x_tensor = torch.tensor(x, dtype=torch.float32).permute(0, 3, 1, 2)
    y_tensor = torch.tensor(y, dtype=torch.long)
    loader = DataLoader(TensorDataset(x_tensor, y_tensor), batch_size=batch_size, shuffle=train_mode)

    total_loss = 0.0
    correct = 0
    total = 0

    if train_mode:
        model.train()
    else:
        model.eval()

    with torch.set_grad_enabled(train_mode):
        for batch_x, batch_y in loader:
            batch_x, batch_y = batch_x.to(device), batch_y.to(device)
            outputs = model(batch_x)
            loss = loss_fn(outputs, batch_y)

            if train_mode:
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            total_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            correct += (preds == batch_y).sum().item()
            total += batch_y.size(0)

    accuracy = correct / total
    avg_loss = total_loss / len(loader)
    return avg_loss, accuracy

# === 7. TRAINING LOOP ===
for epoch in range(epochs):
    train_loss, train_acc = evaluate_model(x_train, y_train, model, loss_fn, device, batch_size, train_mode=True)
    val_loss, val_acc = evaluate_model(x_val, y_val, model, loss_fn, device, batch_size, train_mode=False)

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.4f}")

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "train_accuracy": train_acc,
        "val_loss": val_loss,
        "val_accuracy": val_acc,
    })

# === 8. SAVE MODEL ===
torch.save(model.state_dict(), "decomposed_eval_model.pth")
wandb.save("decomposed_eval_model.pth")


epoch,▁▂▃▃▄▅▆▆▇█
test_samples,▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▃▄▆▇▇▇███
train_loss,█▆▅▄▃▃▂▂▁▁
val_accuracy,▁▃▆▇▆██▅▇█
val_loss,█▆▅▃▄▂▂▄▁▁
epoch,10
test_samples,7178
train_accuracy,0.3637
train_loss,1.63997
val_accuracy,0.36242


Epoch 1/10
  Train Loss: 1.8498 | Train Acc: 0.2513
  Val   Loss: 1.8131 | Val   Acc: 0.2513
  Test  Samples: 7178 (held out for prediction)
Epoch 2/10
  Train Loss: 1.8111 | Train Acc: 0.2513
  Val   Loss: 1.8053 | Val   Acc: 0.2513
  Test  Samples: 7178 (held out for prediction)
Epoch 3/10
  Train Loss: 1.8042 | Train Acc: 0.2513
  Val   Loss: 1.7984 | Val   Acc: 0.2513
  Test  Samples: 7178 (held out for prediction)
Epoch 4/10
  Train Loss: 1.7966 | Train Acc: 0.2511
  Val   Loss: 1.7904 | Val   Acc: 0.2517
  Test  Samples: 7178 (held out for prediction)
Epoch 5/10
  Train Loss: 1.7879 | Train Acc: 0.2528
  Val   Loss: 1.7812 | Val   Acc: 0.2564
  Test  Samples: 7178 (held out for prediction)
Epoch 6/10
  Train Loss: 1.7764 | Train Acc: 0.2585
  Val   Loss: 1.7671 | Val   Acc: 0.2593
  Test  Samples: 7178 (held out for prediction)
Epoch 7/10
  Train Loss: 1.7609 | Train Acc: 0.2695
  Val   Loss: 1.7505 | Val   Acc: 0.2928
  Test  Samples: 7178 (held out for prediction)
Epoch 8/10
  

['/content/wandb/run-20250530_170113-vql4x0yr/files/decomposed_eval_model.pth']

# 1.3 Training FC_3

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb

# === 1. WANDB INIT ===
wandb.init(project="ML_4", entity="nkhar21-student", name="FC_1")

# === 2. CONFIG ===
batch_size = 64
epochs = 20
learning_rate = 0.01
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === 3. MODEL ===
input_dim = 48 * 48
hidden1 = 256
hidden2 = 64
hidden3 = 28  # 7*4
output_dim = 7

model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(input_dim, hidden1),
    nn.ReLU(),
    nn.Linear(hidden1, hidden2),
    nn.ReLU(),
    nn.Linear(hidden2, hidden3),
    nn.ReLU(),
    nn.Linear(hidden3, output_dim),
)
model.to(device)
wandb.watch(model)

# === 4. LOSS + OPTIMIZER ===
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# === 5. TRAINING LOOP ===
for epoch in range(epochs):
    train_loss, train_acc = evaluate_model(x_train, y_train, model, loss_fn, device, batch_size, train_mode=True)
    val_loss, val_acc = evaluate_model(x_val, y_val, model, loss_fn, device, batch_size, train_mode=False)

    # Overfitting metrics
    loss_gap = train_loss - val_loss
    acc_gap = train_acc - val_acc

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.4f}")
    print(f"  Loss Gap:   {loss_gap:.4f} | Acc  Gap:  {acc_gap:.4f}")

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "train_accuracy": train_acc,
        "val_loss": val_loss,
        "val_accuracy": val_acc,
        "loss_gap": loss_gap,
        "acc_gap": acc_gap,
    })

# === 6. SAVE MODEL ===
torch.save(model.state_dict(), "decomposed_eval_model.pth")
wandb.save("decomposed_eval_model.pth")


epoch,▁▂▃▃▄▅▆▆▇█
test_samples,▁▁▁▁▁▁▁▁▁▁
train_accuracy,▁▁▁▁▁▂▃▅▇█
train_loss,█▆▆▅▅▅▄▃▂▁
val_accuracy,▁▁▁▁▁▂▄▄▇█
val_loss,██▇▇▆▅▄▃▃▁
epoch,10
test_samples,7178
train_accuracy,0.32756
train_loss,1.70214
val_accuracy,0.34326


Epoch 1/20
  Train Loss: 1.8756 | Train Acc: 0.2264
  Val   Loss: 1.8181 | Val   Acc: 0.2513
  Loss Gap:   0.0574 | Acc  Gap:  -0.0249
Epoch 2/20
  Train Loss: 1.8124 | Train Acc: 0.2513
  Val   Loss: 1.8083 | Val   Acc: 0.2513
  Loss Gap:   0.0041 | Acc  Gap:  0.0000
Epoch 3/20
  Train Loss: 1.8057 | Train Acc: 0.2513
  Val   Loss: 1.8007 | Val   Acc: 0.2513
  Loss Gap:   0.0050 | Acc  Gap:  0.0000
Epoch 4/20
  Train Loss: 1.7981 | Train Acc: 0.2514
  Val   Loss: 1.7945 | Val   Acc: 0.2499
  Loss Gap:   0.0036 | Acc  Gap:  0.0014
Epoch 5/20
  Train Loss: 1.7900 | Train Acc: 0.2528
  Val   Loss: 1.7878 | Val   Acc: 0.2517
  Loss Gap:   0.0022 | Acc  Gap:  0.0011
Epoch 6/20
  Train Loss: 1.7795 | Train Acc: 0.2587
  Val   Loss: 1.7722 | Val   Acc: 0.2597
  Loss Gap:   0.0072 | Acc  Gap:  -0.0009
Epoch 7/20
  Train Loss: 1.7631 | Train Acc: 0.2710
  Val   Loss: 1.7599 | Val   Acc: 0.2614
  Loss Gap:   0.0033 | Acc  Gap:  0.0096
Epoch 8/20
  Train Loss: 1.7418 | Train Acc: 0.2926
  Val   

['/content/wandb/run-20250530_171029-uxv5ud0o/files/decomposed_eval_model.pth']

# 1.4 Trining FC_4

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
import wandb

# === 1. WANDB INIT ===
wandb.init(project="ML_4", entity="nkhar21-student", name="FC_4")

# === 2. CONFIG ===
batch_size = 64
epochs = 20
learning_rate = 0.005  # changed
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# === 3. MODEL (4 hidden layers) ===
input_dim = 48 * 48
hidden1 = 512
hidden2 = 256
hidden3 = 128
hidden4 = 64
output_dim = 7

model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(input_dim, hidden1),
    nn.ReLU(),
    nn.Linear(hidden1, hidden2),
    nn.ReLU(),
    nn.Linear(hidden2, hidden3),
    nn.ReLU(),
    nn.Linear(hidden3, hidden4),
    nn.ReLU(),
    nn.Linear(hidden4, output_dim),
)
model.to(device)
wandb.watch(model)

# === 4. LOSS + OPTIMIZER ===
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

# === 5. TRAINING LOOP ===
for epoch in range(epochs):
    train_loss, train_acc = evaluate_model(x_train, y_train, model, loss_fn, device, batch_size, train_mode=True)
    val_loss, val_acc = evaluate_model(x_val, y_val, model, loss_fn, device, batch_size, train_mode=False)

    # Overfitting metrics
    loss_gap = train_loss - val_loss
    acc_gap = train_acc - val_acc
    loss_ratio = train_loss / val_loss if val_loss != 0 else float("inf")
    acc_ratio = train_acc / val_acc if val_acc != 0 else float("inf")

    print(f"Epoch {epoch+1}/{epochs}")
    print(f"  Train Loss: {train_loss:.4f} | Train Acc: {train_acc:.4f}")
    print(f"  Val   Loss: {val_loss:.4f} | Val   Acc: {val_acc:.4f}")
    print(f"  Loss Gap:   {loss_gap:.4f} | Acc  Gap:  {acc_gap:.4f}")
    print(f"  Loss Ratio: {loss_ratio:.4f} | Acc Ratio: {acc_ratio:.4f}")

    wandb.log({
        "epoch": epoch + 1,
        "train_loss": train_loss,
        "train_accuracy": train_acc,
        "val_loss": val_loss,
        "val_accuracy": val_acc,
        "loss_gap": loss_gap,
        "acc_gap": acc_gap,
        "loss_ratio": loss_ratio,
        "acc_ratio": acc_ratio,
    })

# === 6. SAVE MODEL ===
torch.save(model.state_dict(), "decomposed_eval_model.pth")
wandb.save("decomposed_eval_model.pth")


acc_gap,▁▄▄▄▄▄▅▂▄▆▅█▃▄▃▃▇▄▄▆
epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss_gap,█▄▄▄▄▄▄▄▅▃▄▁▄▄▄▄▂▄▄▃
train_accuracy,▁▂▂▂▂▃▃▄▅▆▆▇▇▇▇▇████
train_loss,█▆▆▆▆▅▅▄▄▃▃▃▂▂▂▂▁▁▁▁
val_accuracy,▁▁▁▁▁▂▂▅▅▄▆▄▇▆▇█▆██▇
val_loss,██▇▇▇▆▆▅▄▄▃▄▃▂▂▂▃▁▁▁
acc_gap,0.01882
epoch,20
loss_gap,-0.01577
train_accuracy,0.36174


Epoch 1/20
  Train Loss: 1.9079 | Train Acc: 0.2339
  Val   Loss: 1.8804 | Val   Acc: 0.2513
  Loss Gap:   0.0275 | Acc  Gap:  -0.0174
  Loss Ratio: 1.0146 | Acc Ratio: 0.9307
Epoch 2/20
  Train Loss: 1.8581 | Train Acc: 0.2513
  Val   Loss: 1.8366 | Val   Acc: 0.2513
  Loss Gap:   0.0214 | Acc  Gap:  0.0000
  Loss Ratio: 1.0117 | Acc Ratio: 1.0000
Epoch 3/20
  Train Loss: 1.8260 | Train Acc: 0.2513
  Val   Loss: 1.8167 | Val   Acc: 0.2513
  Loss Gap:   0.0092 | Acc  Gap:  0.0000
  Loss Ratio: 1.0051 | Acc Ratio: 1.0000
Epoch 4/20
  Train Loss: 1.8156 | Train Acc: 0.2513
  Val   Loss: 1.8121 | Val   Acc: 0.2513
  Loss Gap:   0.0035 | Acc  Gap:  0.0000
  Loss Ratio: 1.0019 | Acc Ratio: 1.0000
Epoch 5/20
  Train Loss: 1.8129 | Train Acc: 0.2513
  Val   Loss: 1.8105 | Val   Acc: 0.2513
  Loss Gap:   0.0024 | Acc  Gap:  0.0000
  Loss Ratio: 1.0013 | Acc Ratio: 1.0000
Epoch 6/20
  Train Loss: 1.8115 | Train Acc: 0.2513
  Val   Loss: 1.8093 | Val   Acc: 0.2513
  Loss Gap:   0.0023 | Acc  Gap

['/content/wandb/run-20250530_171505-wmvuqj7a/files/decomposed_eval_model.pth']

In [ ]:
x_train[3].min()

np.float32(0.078431375)